# Valorant Round Timer Extraction with TensorFlow Lite Model Maker

This notebook trains a TensorFlow Lite model to extract round timer values from Valorant screenshots.

In [ ]:
## Overview

This notebook will:
1. Train an image classification model to recognize timer values from Valorant screenshots
2. Export the model as TensorFlow Lite for on-device inference
3. The model can classify cropped timer regions into specific time values (e.g., "1:14", "0:45", etc.)

## Dataset Structure

Your training data should be organized as follows:
```
data/
  train/
    1_14/     # Images showing timer at 1:14
      image1.jpg
      image2.jpg
      ...
    1_13/     # Images showing timer at 1:13
      image1.jpg
      ...
    0_45/     # Images showing timer at 0:45
      ...
  test/
    1_14/
      ...
    1_13/
      ...
```

**Note:** Use underscores instead of colons in folder names (e.g., `1_14` instead of `1:14`)

## Prerequisites

### Install Required Packages

Install TensorFlow Lite Model Maker and required dependencies.

### Import Required Libraries


Import TensorFlow and Model Maker libraries for image classification.

In [ ]:
# Install TensorFlow Lite Model Maker
%pip install -q tflite-model-maker
%pip install -q pillow

Now let's import the necessary libraries.

In [ ]:
import numpy as np
import os
from pathlib import Path

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.image_classifier import DataLoader

import tensorflow as tf
assert tf.__version__.startswith('2')
tf.get_logger().setLevel('ERROR')

print(f"TensorFlow version: {tf.__version__}")

## Step 0: Crop Timer Regions from Screenshots

Before training, you need to crop the timer regions from your full Valorant screenshots.

Use the `crop_timer.py` script or the functions below to automatically detect and crop timer regions.


In [ ]:
# Install OpenCV for image processing
%pip install -q opencv-python matplotlib


In [ ]:
# Import the timer cropper
import sys
sys.path.append('.')
from crop_timer_notebook import crop_timer_region, batch_crop_timers, TimerCropper

print("✓ Timer cropper loaded")


### Test on a Single Screenshot

Test the cropping on a single screenshot to verify it works correctly.


In [ ]:
# Test cropping on screenshot.png
test_image = 'screenshot.png'

if os.path.exists(test_image):
    print(f"Testing crop timer on: {test_image}")
    # Crop and show preview
    cropped = crop_timer_region(
        test_image, 
        show_preview=True,  # Shows original with box + cropped result
        method='heuristic'  # Uses fixed position (timer is always in same place)
    )
    if cropped is not None:
        print(f"✓ Cropped image shape: {cropped.shape}")
        print(f"✓ Successfully cropped timer region!")
    else:
        print("✗ Failed to crop timer region")
else:
    print(f"⚠️  Test image not found: {test_image}")


### Batch Crop All Screenshots

Crop timer regions from all screenshots in a directory.


In [ ]:
# Batch crop all screenshots
input_screenshots_dir = 'screenshots'  # Folder with your full screenshots
output_cropped_dir = 'cropped_timers'  # Output folder for cropped images

if os.path.exists(input_screenshots_dir):
    # Count screenshots
    image_files = [f for f in os.listdir(input_screenshots_dir) 
                   if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]
    
    if len(image_files) > 0:
        print(f"Found {len(image_files)} screenshot(s) to process...")
        success_count = batch_crop_timers(
            input_screenshots_dir,
            output_cropped_dir,
            method='heuristic'  # Uses fixed position (fastest and most reliable)
        )
        print(f"\n✓ Cropped {success_count} timer regions")
        print(f"  Output folder: {output_cropped_dir}")
        print(f"  Next: Organize cropped images into data/train/ and data/test/ folders by time value")
    else:
        print(f"⚠️  No screenshots found in {input_screenshots_dir}")
        print("   Add your Valorant screenshots to the 'screenshots/' folder and run this cell again")
else:
    print(f"⚠️  Screenshots directory not found: {input_screenshots_dir}")
    print("   The 'screenshots/' folder has been created - add your Valorant screenshots there")


### Adjust Detection Parameters (if needed)

If the automatic detection doesn't work well for your screenshots, you can adjust the detection parameters.


In [ ]:
# Create a custom cropper with adjusted parameters
from crop_timer_notebook import TimerCropper
import cv2

# Adjust output size if needed (width, height)
custom_cropper = TimerCropper(output_size=(250, 120))  # Larger output

# Test with custom settings
test_image = 'screenshots/screenshot1.jpg'  # Update this path

if os.path.exists(test_image):
    cropped = custom_cropper.crop_timer(
        test_image,
        method='heuristic',  # Use 'heuristic' for consistent positioning
        show_preview=True
    )
else:
    print("⚠️  Test image not found")


## Load Your Training Data

**Important:** The model trains on **cropped timer regions**, not full screenshots!

**Workflow:**
1. Use the cropping functions above (Step 0) to extract timer regions from full screenshots
2. Organize the cropped timer images into folders by time value
3. Train the model on these cropped images

**Data structure:**
- `data/train/` - Training images organized by time value folders
- `data/test/` - Test images organized by time value folders

Each folder should contain cropped timer screenshots showing that specific time value (e.g., `1_14/`, `0_45/`).

**Current crop settings:** 12% width, 6% height, output 400×200px (optimized zoom)

In [ ]:
# Step 1: Crop screenshots (if you have full screenshots)
# Skip this if you already have cropped timer images organized in folders

SCREENSHOTS_DIR = 'screenshots'  # Folder with full Valorant screenshots
CROPPED_DIR = 'cropped_timers'   # Output folder for cropped timer regions

# Set to True if you need to crop screenshots first
NEED_TO_CROP = False  # Change to True if you have screenshots to crop

if NEED_TO_CROP:
    if os.path.exists(SCREENSHOTS_DIR):
        print("Step 1: Cropping timer regions from screenshots...")
        print(f"  Input: {SCREENSHOTS_DIR}")
        print(f"  Output: {CROPPED_DIR}")
        success_count = batch_crop_timers(SCREENSHOTS_DIR, CROPPED_DIR, method='heuristic')
        print(f"\n✓ Cropped {success_count} timer regions")
        print(f"  Next: Organize these cropped images into data/train/ and data/test/ folders")
    else:
        print(f"⚠️  Screenshots directory not found: {SCREENSHOTS_DIR}")
        print("   Set NEED_TO_CROP = False if you already have cropped images organized")
else:
    print("✓ Skipping crop step (assuming you already have cropped timer images)")

# Step 2: Set your data directory path
# This should point to folders containing cropped timer images organized by time value
DATA_DIR = 'data'  # Change this to your actual data path

# Check if data directory exists
print(f"\nStep 2: Checking training data directory...")
if not os.path.exists(DATA_DIR):
    print(f"⚠️  Data directory '{DATA_DIR}' not found!")
    print("\nPlease organize your cropped timer images as follows:")
    print(f"  {DATA_DIR}/train/1_14/    # Images showing timer at 1:14")
    print(f"  {DATA_DIR}/train/1_13/    # Images showing timer at 1:13")
    print(f"  {DATA_DIR}/train/0_45/    # Images showing timer at 0:45")
    print(f"  ...")
    print(f"  {DATA_DIR}/test/1_14/     # Test images for 1:14")
    print(f"  ...")
    print("\nTip: Use batch_crop_timers() above to crop your screenshots first!")
else:
    print(f"✓ Found data directory: {DATA_DIR}")
    # List available classes
    train_dir = os.path.join(DATA_DIR, 'train')
    if os.path.exists(train_dir):
        classes = sorted([d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))])
        print(f"✓ Found {len(classes)} time classes: {classes[:10]}{'...' if len(classes) > 10 else ''}")
        print(f"  These are the timer values your model will learn to classify")
    else:
        print(f"⚠️  Training directory not found: {train_dir}")

## Quickstart: Train Your Timer Classification Model

Follow these steps to train your model:


In [ ]:
# Step 1: Choose a model architecture
# We'll use EfficientNet-Lite0 which is optimized for mobile devices
spec = model_spec.get('efficientnet_lite0')

print(f"✓ Using model: {spec.name}")
print(f"  Input image size: {spec.input_image_shape}")

## Quickstart

There are five steps to train a text classification model:

**Step 1. Choose a text classification model architecture.**

Here we use the average word embedding model architecture, which will produce a small and fast model with decent accuracy.

In [ ]:
# Load training and test data
train_data = DataLoader.from_folder(
    os.path.join(DATA_DIR, 'train'),
    model_spec=spec
)

test_data = DataLoader.from_folder(
    os.path.join(DATA_DIR, 'test'),
    model_spec=spec
)

print(f"✓ Loaded {len(train_data)} training images")
print(f"✓ Loaded {len(test_data)} test images")
print(f"✓ Number of classes: {len(train_data.index_to_label)}")
print(f"  Classes: {list(train_data.index_to_label.values())}")

**Step 3: Train the model**

Train the image classification model on your timer screenshots.

**Step 4: Evaluate the model**

Test the model's accuracy on your test dataset.

In [ ]:
# Train the model
# Adjust epochs based on your dataset size and desired accuracy
# More epochs = better accuracy but longer training time
model = image_classifier.create(
    train_data,
    model_spec=spec,
    epochs=10,  # Start with 10, increase if needed
    batch_size=32,
    validation_data=test_data
)

print("✓ Model training completed!")

**Step 5: Export as TensorFlow Lite model**

Export the trained model in TensorFlow Lite format for on-device inference.

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_data)
print(f"✓ Test accuracy: {accuracy:.2%}")
print(f"✓ Test loss: {loss:.4f}")

## Export the Model

Export the trained model as a TensorFlow Lite file for deployment.

In [ ]:
# Export the model
export_dir = 'valorant_timer_model'
model.export(export_dir=export_dir)

print(f"✓ Model exported to: {export_dir}/model.tflite")
print(f"✓ Model size: {os.path.getsize(os.path.join(export_dir, 'model.tflite')) / 1024:.2f} KB")

## Test the Model

Test the exported model on a sample image to verify it works correctly.

In [ ]:
# Test the model on a sample image
from PIL import Image
import matplotlib.pyplot as plt

# Load a test image (replace with path to your test image)
test_image_path = None  # Set this to a test image path

if test_image_path and os.path.exists(test_image_path):
    # Load and display the image
    img = Image.open(test_image_path)
    plt.figure(figsize=(8, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Test Image')
    plt.show()
    
    # Run inference
    result = model.evaluate_tflite(os.path.join(export_dir, 'model.tflite'), test_data)
    print(f"✓ TFLite model accuracy: {result:.2%}")
else:
    print("⚠️  Set test_image_path to test the model on a specific image")

You can download the TensorFlow Lite model file using the left sidebar of Colab. Go into the `average_word_vec` folder as we specified in `export_dir` parameter above, right-click on the `model.tflite` file and choose `Download` to download it to your local computer.

This model can be integrated into an Android or an iOS app using the [NLClassifier API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/nl_classifier) of the [TensorFlow Lite Task Library](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview).

See the [TFLite Text Classification sample app](https://github.com/tensorflow/examples/blob/master/lite/examples/text_classification/android/lib_task_api/src/main/java/org/tensorflow/lite/examples/textclassification/client/TextClassificationClient.java#L54) for more details on how the model is used in a working app.

*Note 1: Android Studio Model Binding does not support text classification yet so please use the TensorFlow Lite Task Library.*

*Note 2: There is a `model.json` file in the same folder with the TFLite model. It contains the JSON representation of the [metadata](https://www.tensorflow.org/lite/models/convert/metadata) bundled inside the TensorFlow Lite model. Model metadata helps the TFLite Task Library know what the model does and how to pre-process/post-process data for the model. You don't need to download the `model.json` file as it is only for informational purpose and its content is already inside the TFLite file.*

*Note 3: If you train a text classification model using MobileBERT or BERT-Base architecture, you will need to use [BertNLClassifier API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_nl_classifier) instead to integrate the trained model into a mobile app.*

The following sections walk through the example step by step to show more details.

**Step 6: Use `TFLite Task Library` to demo how to use the trained models**

Read the dev.csv file into the sentence data to predict with the trained model

In [ ]:
sentence_data = pd.read_csv('/content/dev.csv', index_col=0)
sentence_data

Model config parameter

In [ ]:
# Name of the TFLite text classification model.
_MODEL = '/content/average_word_vec/model.tflite'
# Whether to run the model on EdgeTPU.
_ENABLE_EDGETPU = False
# Number of CPU threads to run the model.
_NUM_THREADS = 4

Initialize model

We could also change the parameters like `file_name`, `use_coral`, and `num_threads` that could affect the model results. The parameters you can adjust are:

*   `file_name`: Name of the TFLite image classification model.
*   `use_coral`: If true, inference will be delegated to a connected Coral Edge TPU device.
*   `num_threads`: Number of CPU threads to run the model.

In [ ]:
# Initialize the text classification model.
base_options = core.BaseOptions(file_name=_MODEL, use_coral=_ENABLE_EDGETPU, num_threads=_NUM_THREADS)
options = text.NLClassifierOptions(base_options)

# Create NLClassifier from options.
classifier = text.NLClassifier.create_from_options(options)

Predict using `TFLite Task Library`

In [ ]:
for idx in range(20):
  sentence = sentence_data['sentence'].iloc[idx]
  label = sentence_data['label'].iloc[idx]
  text_classification_result = classifier.classify(sentence)
  classification_list = text_classification_result.classifications[0].categories

  # Sort output by probability descending.
  predict_label = sorted(
      classification_list, key=lambda item: item.score, reverse=True)[0]

  print('truth_label: {} -----> predict_label: {}'.format(label, predict_label.category_name))

## Choose a model architecture for Text Classifier

Each `model_spec` object represents a specific model for the text classifier. TensorFlow Lite Model Maker currently supports [MobileBERT](https://arxiv.org/pdf/2004.02984.pdf), averaging word embeddings and [BERT-Base](https://arxiv.org/pdf/1810.04805.pdf) models.

| Supported Model          | Name of model_spec      | Model Description                                                                                                     | Model size                                  |
|--------------------------|-------------------------|-----------------------------------------------------------------------------------------------------------------------|---------------------------------------------|
| Averaging Word Embedding | 'average_word_vec'      | Averaging text word embeddings with RELU activation.                                                                  |           <1MB                             |
| MobileBERT               | 'mobilebert_classifier' | 4.3x smaller and 5.5x faster than BERT-Base while achieving competitive results, suitable for on-device applications. | 25MB w/ quantization <br/> 100MB w/o quantization                                        |
| BERT-Base                | 'bert_classifier'       | Standard BERT model that is widely used in NLP tasks.                                                                 | 300MB |

In the quick start, we have used the average word embedding model. Let's switch to [MobileBERT](https://arxiv.org/pdf/2004.02984.pdf) to train a model with higher accuracy.

In [ ]:
mb_spec = model_spec.get('mobilebert_classifier')

## Load training data

You can upload your own dataset to work through this tutorial. Upload your dataset by using the left sidebar in Colab.

<img src="https://storage.googleapis.com/download.tensorflow.org/models/tflite/screenshots/model_maker_text_classification.png" alt="Upload File" width="800" hspace="100">

If you prefer not to upload your dataset to the cloud, you can also locally run the library by following the [guide](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker).

To keep it simple, we will reuse the SST-2 dataset downloaded earlier. Let's use the `DataLoader.from_csv` method to load the data.

Please be noted that as we have changed the model architecture, we will need to reload the training and test dataset to apply the new preprocessing logic.

In [ ]:
train_data = DataLoader.from_csv(
      filename='train.csv',
      text_column='sentence',
      label_column='label',
      model_spec=mb_spec,
      is_training=True)
test_data = DataLoader.from_csv(
      filename='dev.csv',
      text_column='sentence',
      label_column='label',
      model_spec=mb_spec,
      is_training=False)

The Model Maker library also supports the `from_folder()` method to load data. It assumes that the text data of the same class are in the same subdirectory and that the subfolder name is the class name. Each text file contains one movie review sample. The `class_labels` parameter is used to specify which the subfolders.

## Train a TensorFlow Model

Train a text classification model using the training data.

*Note: As MobileBERT is a complex model, each training epoch will takes about 10 minutes on a Colab GPU. Please make sure that you are using a GPU runtime.*

In [ ]:
model = text_classifier.create(train_data, model_spec=mb_spec, epochs=3)

Examine the detailed model structure.

In [ ]:
model.summary()

## Evaluate the model

Evaluate the model that we have just trained using the test data and measure the loss and accuracy value.

In [ ]:
loss, acc = model.evaluate(test_data)

## Export as a TensorFlow Lite model

Convert the trained model to TensorFlow Lite model format with [metadata](https://www.tensorflow.org/lite/models/convert/metadata) so that you can later use in an on-device ML application. The label file and the vocab file are embedded in metadata. The default TFLite filename is `model.tflite`.

In many on-device ML application, the model size is an important factor. Therefore, it is recommended that you apply quantize the model to make it smaller and potentially run faster.
The default post-training quantization technique is dynamic range quantization for the BERT and MobileBERT models.

In [ ]:
model.export(export_dir='mobilebert/')

The TensorFlow Lite model file can be integrated in a mobile app using the [BertNLClassifier API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_nl_classifier) in [TensorFlow Lite Task Library](https://www.tensorflow.org/lite/inference_with_metadata/task_library/overview). Please note that this is **different** from the `NLClassifier` API used to integrate the text classification trained with the average word vector model architecture.

The export formats can be one or a list of the following:

*   `ExportFormat.TFLITE`
*   `ExportFormat.LABEL`
*   `ExportFormat.VOCAB`
*   `ExportFormat.SAVED_MODEL`

By default, it exports only the TensorFlow Lite model file containing the model metadata. You can also choose to export other files related to the model for better examination. For instance, exporting only the label file and vocab file as follows:

In [ ]:
model.export(export_dir='mobilebert/', export_format=[ExportFormat.LABEL, ExportFormat.VOCAB])

You can evaluate the TFLite model with `evaluate_tflite` method to measure its accuracy. Converting the trained TensorFlow model to TFLite format and apply quantization can affect its accuracy so it is recommended to evaluate the TFLite model accuracy before deployment.

In [ ]:
accuracy = model.evaluate_tflite('mobilebert/model.tflite', test_data)
print('TFLite model accuracy: ', accuracy)

## Advanced Usage

The `create` function is the driver function that the Model Maker library uses to create models. The `model_spec` parameter defines the model specification. The `AverageWordVecSpec` and `BertClassifierSpec` classes are currently supported. The `create` function comprises of the following steps:

1. Creates the model for the text classifier according to `model_spec`.
2. Trains the classifier model.  The default epochs and the default batch size are set by the `default_training_epochs` and `default_batch_size` variables in the `model_spec` object.

This section covers advanced usage topics like adjusting the model and the training hyperparameters.

### Customize the MobileBERT model hyperparameters

The model parameters you can adjust are:

* `seq_len`: Length of the sequence to feed into the model.
* `initializer_range`: The standard deviation of the `truncated_normal_initializer` for initializing all weight matrices.
* `trainable`: Boolean that specifies whether the pre-trained layer is trainable.

The training pipeline parameters you can adjust are:

* `model_dir`: The location of the model checkpoint files. If not set, a temporary directory will be used.
* `dropout_rate`: The dropout rate.
* `learning_rate`: The initial learning rate for the Adam optimizer.
* `tpu`: TPU address to connect to.

For instance, you can set the `seq_len=256` (default is 128). This allows the model to classify longer text.

In [ ]:
new_model_spec = model_spec.get('mobilebert_classifier')
new_model_spec.seq_len = 256

### Customize the average word embedding model hyperparameters

You can adjust the model infrastructure like the `wordvec_dim` and the `seq_len` variables in the `AverageWordVecSpec` class.


For example, you can train the model with a larger value of `wordvec_dim`. Note that you must construct a new `model_spec` if you modify the model.

In [ ]:
new_model_spec = AverageWordVecSpec(wordvec_dim=32)

Get the preprocessed data.

In [ ]:
new_train_data = DataLoader.from_csv(
      filename='train.csv',
      text_column='sentence',
      label_column='label',
      model_spec=new_model_spec,
      is_training=True)

Train the new model.

In [ ]:
model = text_classifier.create(new_train_data, model_spec=new_model_spec)

### Tune the training hyperparameters
You can also tune the training hyperparameters like `epochs` and `batch_size` that affect the model accuracy. For instance,

*   `epochs`: more epochs could achieve better accuracy, but may lead to overfitting.
*   `batch_size`: the number of samples to use in one training step.

For example, you can train with more epochs.

In [ ]:
model = text_classifier.create(new_train_data, model_spec=new_model_spec, epochs=20)

Evaluate the newly retrained model with 20 training epochs.

In [ ]:
new_test_data = DataLoader.from_csv(
      filename='dev.csv',
      text_column='sentence',
      label_column='label',
      model_spec=new_model_spec,
      is_training=False)

loss, accuracy = model.evaluate(new_test_data)

### Change the Model Architecture

You can change the model by changing the `model_spec`. The following shows how to change to BERT-Base model.

Change the `model_spec` to BERT-Base model for the text classifier.

In [ ]:
spec = model_spec.get('bert_classifier')

The remaining steps are the same.

### Customize Post-training quantization on the TensorFlow Lite model

[Post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization) is a conversion technique that can reduce model size and inference latency, while also improving CPU and hardware accelerator inference speed, with a little degradation in model accuracy. Thus, it's widely used to optimize the model.

Model Maker library applies a default post-training quantization techique when exporting the model. If you want to customize post-training quantization, Model Maker supports multiple post-training quantization options using [QuantizationConfig](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/config/QuantizationConfig) as well. Let's take float16 quantization as an instance. First, define the quantization config.

```python
config = QuantizationConfig.for_float16()
```


Then we export the TensorFlow Lite model with such configuration.

```python
model.export(export_dir='.', tflite_filename='model_fp16.tflite', quantization_config=config)
```

# Read more

You can read our [text classification](https://www.tensorflow.org/lite/examples/text_classification/overview) example to learn technical details. For more information, please refer to:

*   TensorFlow Lite Model Maker [guide](https://www.tensorflow.org/lite/models/modify/model_maker) and [API reference](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker).
*  Task Library: [NLClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/nl_classifier) and [BertNLClassifier](https://www.tensorflow.org/lite/inference_with_metadata/task_library/bert_nl_classifier) for deployment.
*   The end-to-end reference apps: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/text_classification/android) and [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/text_classification/ios).